In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
tf.random.set_seed(42)

pd.set_option('display.max_columns', 500) 

# Load Data: DB1(2020.5.17).xlsx

In [2]:
#data_jaun = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'digestive(jaundice)')
#data_hemate = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'hematemesis(digestive)')
#data_Folie_Nerve = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'Folie_Nerve')
#data_musculoskeletal = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'musculoskeletal')
#data_kidney = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'kidney')
#data_respiratory = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'respiratory')
#data_circulator1 = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'circulator1')
#data_circulator2 = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'Circulator2')
#data_para = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'Para_Female_Child')
#data_digestive = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'digestive')
#others = pd.read_excel('결측치db.xls', sheet_name='Sheet2')

In [3]:
data_jaun = pd.read_excel('DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'digestive(jaundice)')
data_hemate = pd.read_excel('DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'hematemesis(digestive)')
#data_Folie_Nerve = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'Folie_Nerve')
data_musculoskeletal = pd.read_excel('DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'musculoskeletal')
data_kidney = pd.read_excel('DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'kidney')
data_respiratory = pd.read_excel('DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'respiratory')
data_circulator1 = pd.read_excel('DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'circulator1')
data_circulator2 = pd.read_excel('DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'Circulator2')
data_para = pd.read_excel('DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'Para_Female_Child')
data_digestive = pd.read_excel('DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'digestive')
#others = pd.read_excel('결측치db.xls', sheet_name='Sheet2')

# 1. Preprocessing

## 0) Select columns and fill diseases

In [4]:
def organize_data(df):
  df.columns = df.iloc[0]
  df = df[1:]
  df = df.iloc[:, 6:]
  df['level5/diagnosis'] = df['level5/diagnosis'].fillna(method='ffill') 
  df.drop('No.case', axis=1, inplace=True)

  return df

In [5]:
data_digestive = organize_data(data_digestive)
data_musculoskeletal = organize_data(data_musculoskeletal)
data_kidney = organize_data(data_kidney)
data_respiratory = organize_data(data_respiratory)
data_para = organize_data(data_para)
#data_Folie_Nerve = organize_data(data_Folie_Nerve)
data_circulator1 = organize_data(data_circulator1)
data_circulator1 = data_circulator1.iloc[:, :21]

data_circulator2 = data_circulator2.iloc[:, 1:]
data_circulator2['진단 질병'] = data_circulator2['진단 질병'].fillna(method='ffill')
data_circulator2.drop('No.case', axis=1, inplace=True) 

In [6]:
data_hemate = organize_data(data_hemate)
data_jaun = organize_data(data_jaun)

data_hemate['Level1'] = 'hematemesis(digestive)'
data_jaun['Level1'] = 'digestive(jaundice)'

In [7]:
data_digestive['Level1'] = 'digestive'
data_musculoskeletal['Level1'] = 'musculoskeletal'
data_kidney['Level1'] = 'kidney'
data_respiratory['Level1'] = 'respiratory'
data_para['Level1'] = 'para'
data_circulator1['Level1'] = 'circulator1'
data_circulator2['Level1'] = 'circulator2'

In [8]:
print(data_hemate.shape)
print(data_jaun.shape)
print(data_digestive.shape)
print(data_musculoskeletal.shape)
print(data_kidney.shape)
print(data_respiratory.shape)
print(data_para.shape)
#print(data_Folie_Nerve.shape)
print(data_circulator1.shape)
print(data_circulator2.shape)

(360, 22)
(440, 22)
(421, 22)
(205, 22)
(246, 22)
(205, 22)
(355, 22)
(305, 22)
(218, 22)


In [9]:
data_circulator2.columns = data_circulator1.columns

data = pd.concat([data_digestive, data_musculoskeletal, data_kidney,
                  data_respiratory, data_circulator1, #data_Folie_Nerve,
                  data_circulator2, data_para, data_jaun, data_hemate], axis=0)
data.sample()

,level5/diagnosis,Chief complaint,Sex,Age,Height,Weight,Onset,Location,Duration,Course,Experience,Character,Associated Sx.,Factor,Event,약물 투약력,사회력,가족력,외상력,과거력,여성력,Level1
342,Constitutional delay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,para


In [10]:
nan_data = pd.read_excel('at_least_10_nan_data.xlsx', sheet_name = 'Sheet1')
nan_data.drop('Unnamed: 0', inplace=True, axis=1)
nan_data.head(1)

,level5/diagnosis,Chief complaint,Sex,Age,Height,Weight,Onset,Location,Duration,Course,Experience,Character,Associated Sx.,Factor,Event,약물 투약력,사회력,가족력,외상력,과거력,여성력,Level1
0,Reflux esophagitis,피 토했어요,여자,40,165,65.0,2달전,NaN,NaN,비슷함,처음,"붉은색, 덩어리(-), 음식물(-)","복통, 기침",NaN,특이사항 없음,NaN,NaN,NaN,NaN,NaN,정상 월경주기,digestive


In [11]:
print(data.shape)
data = pd.concat([data, nan_data], axis=0)
print(data.shape)

(2755, 22)
(2953, 22)


## 1) Drop rows if more than half are nan

In [12]:
data = data.fillna('-')

In [13]:
def to_nan(x):
  if(x == '-'):
    x = np.nan
  if(x == '(-)'):
    x = np.nan
  if(x=='없음'):
    x = np.nan
  if(x=='특이사항 없음'):
    x = np.nan
  if(x=='이상 없음'):
    x = np.nan
  if(x=='.'):
    x = np.nan
    
  return x

In [14]:
for i in range(len(data.columns)):
    data[data.columns[i]] = data.apply(lambda x : to_nan(x[data.columns[i]]) , axis = 1 )

In [15]:
print('Before', data.shape)
data.dropna(thresh = data.shape[1]/2, axis = 0, inplace = True)
print('After', data.shape)

Before (2953, 22)
After (2238, 22)


In [16]:
data.reset_index(drop=True, inplace=True)

In [17]:
print(data.shape)
print(data['level5/diagnosis'].value_counts())

(2238, 22)
Rupture of esophageal varices     100
Acute cholecystitis                50
Gastric ulcer / duodenal ulcer     50
Chronic pancreatitis               50
Esophageal cancer                  50
                                 ... 
Chronic renal failure               1
Drug-induced tachycardia            1
  Use of hormonal drugs             1
Transient ischemic attack           1
intraducal papilloma                1
Name: level5/diagnosis, Length: 289, dtype: int64


In [18]:
#idx_num = data[data['Height'].isnull()].index
#data = data.drop(idx_num)

## 2) Change Nan to blank text

In [19]:
data = data.fillna('-')

In [20]:
def to_nan(x):
  if(x == '-'):
    x = ''
  elif(x == '아니오'):
    x = ''
  elif(x == '아뇨'):
    x = ''
  elif(x == '몰라요'):
    x = ''
  elif(x == '모릅니다'):
    x = ''
  elif(x == '모름'):
    x = ''
  elif(x == '아뇨'):
    x = ''
  elif(x == '아뇨'):
    x = ''
  elif(x == '.'):
    x = ''    
  return x

In [21]:
for i in range(len(data.columns)):
    data[data.columns[i]] = data.apply(lambda x : to_nan(x[data.columns[i]]) , axis = 1 )

In [22]:
print(data.shape)

(2238, 22)


## 3) Check Target Data, Disease

In [23]:
data['level5/diagnosis'].value_counts()

Rupture of esophageal varices     100
Acute cholecystitis                50
Gastric ulcer / duodenal ulcer     50
Chronic pancreatitis               50
Esophageal cancer                  50
                                 ... 
Chronic renal failure               1
Drug-induced tachycardia            1
  Use of hormonal drugs             1
Transient ischemic attack           1
intraducal papilloma                1
Name: level5/diagnosis, Length: 289, dtype: int64

In [24]:
len(data['level5/diagnosis'].unique())

289

### Organize differently expressed diseases to the same expression diseases

In [25]:
disease_kor = pd.read_excel('ver6_new_words.xlsx', sheet_name = '질병')
disease_kor = disease_kor[['level5/diagnosis', '번역']]
disease_kor.head()

,level5/diagnosis,번역
0,Pseudoparathyroidism,가성 부갑상선 기능 저하증
1,Familial hypercholesterolemia,가족성 고콜레스테롤혈증
2,Familial short stature,가족형 저신장증
3,hepatitis,간염
4,Interstitial lung disease (ILD),간질성 폐질환


In [26]:
data = pd.merge(data, disease_kor, how='left',on='level5/diagnosis')
data = data.drop_duplicates()
data.reset_index(drop=True, inplace=True)

data['번역'].fillna(data['level5/diagnosis'], inplace=True)
data['level5/diagnosis'] = data['번역']
data.drop('번역', axis=1, inplace=True)
data.sample()

,level5/diagnosis,Chief complaint,Sex,Age,Height,Weight,Onset,Location,Duration,Course,Experience,Character,Associated Sx.,Factor,Event,약물 투약력,사회력,가족력,외상력,과거력,여성력,Level1
538,척추 종양,목 아파요,여자,61,162,53,3개월전,목,하루종일,심해짐,처음,뻐근한 통증,체중 감소,,,,,,,,,musculoskeletal


In [27]:
print(data.shape)
#print(disease_kor.shape)
#print(a.shape)

(2222, 22)


In [28]:
#ind = data[data['level5/diagnosis'] == 'Gastritis, gastric ulcer, duodenal ulcer'].index
#data.loc[ind, data.columns[0]] = 'Gastric ulcer / duodenal ulcer'
#data[data['level5/diagnosis'] == 'Gastric ulcer / duodenal ulcer']

#ind = data[data['level5/diagnosis'] == 'Irritable bowel syndrome (IBS)'].index
#data.loc[ind, data.columns[0]] = 'Irritable bowel syndrome'

#ind = data[data['level5/diagnosis'] == 'Irritable Bowel Syndrome (IBS)'].index
#data.loc[ind, data.columns[0]] = 'Irritable bowel syndrome'

- Change some disease names in Circulator2

In [29]:
idx = data[data['level5/diagnosis'] == '공황장애 / 과다환기증후군'].index#['level5/diagnosis'] = '공황장애'
data.loc[idx, 'level5/diagnosis'] = '공황장애'

idx = data[data['level5/diagnosis'] == '소화성궤양'].index
data.loc[idx, 'level5/diagnosis'] = '소화성 궤양'

idx = data[data['level5/diagnosis'] == '기립성 저혈압(혈량저하증 동반되어 있을 수)'].index
data.loc[idx, 'level5/diagnosis'] = '기립성 저혈압(혈량저하증 동반 가능)'

idx = data[data['level5/diagnosis'] == '대동맥박리'].index
data.loc[idx, 'level5/diagnosis'] = '대동맥 박리'

idx = data[data['level5/diagnosis'] == '역류성식도염'].index
data.loc[idx, 'level5/diagnosis'] = '역류성 식도염'

idx = data[data['level5/diagnosis'] == '거미막하출혈(SAH)'].index
data.loc[idx, 'level5/diagnosis'] = '지주막하 출혈'

idx = data[data['level5/diagnosis'] == '과다환기증'].index # 원문: Panic disorder / hyperventilation syndrome
data.loc[idx, 'level5/diagnosis'] = '공황장애'

idx = data[data['level5/diagnosis'] == '판막 질환(AR, MR 등)'].index
data.loc[idx, 'level5/diagnosis'] = '판막 질환'

idx = data[data['level5/diagnosis'] == '폐경후증후군'].index
data.loc[idx, 'level5/diagnosis'] = '폐경증후군'

idx = data[data['level5/diagnosis'] == '폐경증후군(폐경)'].index
data.loc[idx, 'level5/diagnosis'] = '폐경증후군'

idx = data[data['level5/diagnosis'] == '마주신경성 실신'].index
data.loc[idx, 'level5/diagnosis'] = '미주신경성 실신'

In [30]:
len(data['level5/diagnosis'].unique())

227

### Drop disease which has less than 5 cases

In [31]:
target_counts = data['level5/diagnosis'].value_counts().reset_index()
target_counts.columns = ['Disease', 'counts']
target_counts = target_counts[target_counts['counts'] < 5]
target_counts

,Disease,counts
225,신증후군,4
226,갈색세포종,3


In [32]:
target_counts = data['level5/diagnosis'].value_counts().reset_index()
target_counts.columns = ['Disease', 'counts']
target_counts = target_counts[target_counts['counts'] >= 5]
target_counts

,Disease,counts
0,식도 정맥류 파열,100
1,소화성 궤양,95
2,역류성 식도염,64
3,식도암,50
4,급성 간염,50
...,...,...
220,칸디다 질염,5
221,간질성 폐질환,5
222,감염성 기침,5
223,미주신경성 실신,5


In [33]:
data = data[data['level5/diagnosis'].isin(target_counts['Disease'])]
data.reset_index(drop=True, inplace=True)

In [34]:
print(data.shape)
print(data['level5/diagnosis'].value_counts())

(2215, 22)
식도 정맥류 파열    100
소화성 궤양        95
역류성 식도염       64
식도암           50
급성 간염         50
            ... 
간질성 폐질환        5
감염성 기침         5
미주신경성 실신       5
요붕증            5
크론병            5
Name: level5/diagnosis, Length: 225, dtype: int64


## 4) Calculate BMI and define Obesity with Weight and Height
- https://www.tongyeong.go.kr/health/01635/01898/01902.web

In [35]:
# Correct Height error
idx = data[data['Height'] == '58세 여성이 소변을 자주 보고 소변을 참기 어려우며 소변 누러 화장실을 가면 미처 속옷을 내리기도 전에 소변이 흘러나오는 증상이 4년 이상 지속되어 내원하였다. 기침을 하면 소변이 약간 새며,밤에 자다가도 소변이 마려워 3번 정도 깬다고 한다. 소변이 속옷에 묻어 냄새 및 회음부 주위 피부염이 발생하며 상기 증상으로 사회생활도 힘들다고 한다. 특이 과거력은 없다.'].index
data.loc[idx, 'Height'] = ''
#print(data['Height'].iloc[850])
#data['Height'].iloc[850]=''
#print(data['Height'].iloc[850])

In [36]:
data['Height'] = data['Height'].replace('', '0')
data['Weight'] = data['Weight'].replace('', '0')

data['Height'] = data['Height'].astype('int')
data['Weight'] = data['Weight'].astype('int')

In [37]:
data['BMI'] = data['Weight'] / (data['Height']/100)**2
data['BMI'].fillna(-1, inplace=True)

In [38]:
def define_obesity(x):
  if x < 0:
    return '알 수 없음'
  elif x < 20.0:
    return '저체중'
  elif x <= 24.0:
    return '정상'
  elif x <= 29.0:
    return '과체중'
  else:
    return '비만'

data['Obesity'] = data.apply(lambda x : define_obesity(x['BMI']), axis=1)

## 5) Change Age to Groups of Age

In [39]:
data['Age'] = data['Age'].astype(str)

In [40]:
import re
# Infant/Child's Grouping Reference: https://www.doctorsnews.co.kr/news/articleView.html?idxno=20917&replyAll=&reply_sc_order_by=I

def replace_age(x):

  if ('개월' in x) or ('달' in x) or ('개월' in x):
    #print(x)
    month_age = int(re.findall("\d+", x)[0])*4
    x = '생후 ' + str(month_age) + '주'
  
  if ('주' in x):
     week_age = int(re.findall("\d+", x)[0])
     if week_age <= 4:
       x = '신생아' # 태어나서 1개월까지(newborn)
     elif week_age <= 4*12:
       x = '영아' # 태어나서 1년까지(infant)
     else:
       x = '유아' # 만 1세부터 6세까지(child)
  else:
     x = x.replace('세', '')
     x = x[:2]
     x = int(x)#.astype(int)
     x = str(x//10*10)+'s'

  return x

data['Age'] = data.apply(lambda x : replace_age(x['Age']), axis=1)

In [41]:
data['Age']

0       50s
1       30s
2       70s
3       10s
4       60s
       ... 
2210    50s
2211    30s
2212    60s
2213    60s
2214    10s
Name: Age, Length: 2215, dtype: object

## 6) Arrange Sex

In [42]:
data['Sex'].value_counts()

남자      829
여자      691
남성      139
여성      130
F       129
M        92
남자       79
여자       63
남성       45
여자       15
          3
Name: Sex, dtype: int64

In [43]:
def organize_sex(x):
  x = x.replace('F', '여자')
  x = x.replace('M', '남자')
  x = x.replace('여성', '여자')
  x = x.replace('남성', '남자')
  #x = x.lstrip()
  x = x.replace('여자 ', '여자')
  x = x.replace('여자  ', '여자')
  x = x.replace('남자 ', '남자')
  x = x.replace('남성 ', '남자')

  return x

data['Sex'] = organize_sex(data['Sex'])

In [44]:
data['Sex'].value_counts()

남자    1184
여자    1028
         3
Name: Sex, dtype: int64

## 7) Remove Negative values(-)

In [45]:
data.columns[10:21]

Index(['Experience', 'Character', 'Associated Sx.', 'Factor', 'Event',
       '약물 투약력', '사회력', '가족력', '외상력', '과거력', '여성력'],
      dtype='object')

In [46]:
#k = data.copy()

In [47]:
# since regex doesn't support '+', change '+' to '#'
def to_new_positive_value(x):
  if('+' in x):
    x = x.replace('+', '#')#np.nan
    
  return x

for m in data.columns[10:21]: #'Experience', 'Character', 'Associated Sx.', 'Factor', 'Event', '약물 투약력', '사회력', '가족력', '외상력', '과거력', '여성력']
  data[m] = data.apply(lambda x : to_new_positive_value(x[m]) , axis = 1 )

In [48]:
idx = data[data['Associated Sx.'] == '발열(#, 4주 전) 가래(#), 기침(#), 체중변화 (#, -4kg/2개월)'].index
data.loc[idx, 'Associated Sx.'] = '발열(#, 4주 전), 가래(#), 기침(#), 체중변화 (#, -4kg/2개월)'

idx = data[data['사회력'] == '술(#주) 흡연(#), 과식(#), 지방식(#)'].index
data.loc[idx, '사회력'] = '술(#주), 흡연(#), 과식(#), 지방식(#)'

In [49]:
data['Associated Sx.'][0]

'[전신] 발열 / 식은땀 / 체중 감소 / 오한/ 피로감(-/-/-/-/-), [소화기] 구역 / 구토 / 식욕 부진 / 변비 / 설사(-/#, 2회 흰 물만 나옴 /-/-/-), 속쓰림, 혈변(-), [응급] 어지러움 / 갈증 / 소변량 감소 / 호흡곤란(#/#/#/-)'

In [50]:
def remove_negative_values(x):
  
  #print('x',x,'1')
  # 1. To extract texts from brackets: (-/#, 2회 흰 물만 나옴 /-/-/-).
  brackets = re.findall('\((.*?)\)', x) # ['-/-/-/-/-', '-/#, 2회 흰 물만 나옴 /-/-/-', '-', '#/#/#/-']
  for i in range(len(brackets)):
    n = brackets[i].split('/') # ['-', '#, 2회 흰 물만 나옴 ', '-', '-', '-']

    for j in range(len(n)):
      if len(n[j])>1:
        tmp_str = n[j][2:] #  Behind '#, ' --> ' 2회 흰 물만 나옴 '
        if len(re.findall('\#', n[j]))==0: # '(#) (지금 복용 중)'
          n[j]=''
        else:
          n[j] = '#'   
        x = x.replace(brackets[i],'/'.join(n)) # Drawback: doesn't contain texts which was in brackets

  #if len(re.findall(r"(\d+)-?(\d+)", x)) > 0: # 하루 4-5회
  #  x = x.replace('-', '~')

  # 2. To remove negative values
  splited_x = x.split(',')
  for i in range(len(splited_x)):
    if ('및' in splited_x[i]):
      splited_x = x.split(',') and x.split('및')
      break

  #print(splited_x)
  for i in range(len(splited_x)):
    ne_or_po = re.findall('-|#',splited_x[i]) #.index('') ['#', '#', '#', '-']
    sym = splited_x[i].split('/') # [' [응급] 어지러움 ', ' 갈증 ', ' 소변량 감소 ', ' 호흡곤란(#', '#', '#', '-)']

    only_positive = ""
    #print(ne_or_po)
    for j in range(len(ne_or_po)):
      if ne_or_po[j] != '-':
        only_positive += sym[j]

      splited_x[i] = only_positive
        #print('@fin_splited_x', splited_x[i])
  x = (',').join(splited_x)
  #print('@@fin_x', x)

  return x

#def remove_negative_values(x):
#  tmp = x.split(',')
#  for i in range(len(tmp)):
#    if ('-' in tmp[i]):
#      tmp[i] = ''
#  x = ''.join(tmp)
#  return x
#for i in range(len(data.columns)):
#    data[data.columns[i]] = data.apply(lambda x : remove_negative_values(x[data.columns[i]]) , axis = 1 )

In [51]:
for m in data.columns[10:21]: #'Experience', 'Character', 'Associated Sx.', 'Factor', 'Event', '약물 투약력', '사회력', '가족력', '외상력', '과거력', '여성력']
  data[m] = data.apply(lambda x : remove_negative_values(x[m]) , axis = 1 )

In [52]:
data['Associated Sx.'][0]

', 구토 , 속쓰림,, [응급] 어지러움  갈증  소변량 감소 '

In [53]:
def to_ori_new_positive_value(x):
  if('#' in x):
    x = x.replace('#', '+')#np.nan
    
  return x

for m in data.columns[10:21]: #'Experience', 'Character', 'Associated Sx.', 'Factor', 'Event', '약물 투약력', '사회력', '가족력', '외상력', '과거력', '여성력']
  data[m] = data.apply(lambda x : to_ori_new_positive_value(x[m]) , axis = 1 )

## 8) NRS scores to text

In [54]:
#NRS 5점
#NRS: 5
#NRS 4~6 / NRS : 4~6

In [55]:
def NRS_to_text(text):

  a = re.findall('NRS.*?점', text) #NRS 5점
  if len(a) == 0:
    a = re.findall('NRS.*?\d~\d', text) #NRS 4~6 / NRS : 4~6
  if len(a) == 0:
    a = re.findall('NRS.*?\d', text) #NRS: 5
  
  try:
    t = re.findall('[0-9]+', a[0])

    if (len(t) == 2):
      score = (int(t[0])+int(t[1]))/ 2
    else:
      score = int(t[0])

    if score >= 7:
      to_text = '심함'
    elif score >= 4:
      to_text = '중간'
    else:
      to_text = '약함'

    text = text.replace(a[0], to_text)
  
  except:
    pass

  return text

In [56]:
#text = '뻐근한 통증, NRS : 1~6, 등이랑 목으로 뻗침'
text = '뻐근한 통증dd, NRS : 1~6, 등이랑 목으로 뻗침'

NRS_to_text(text)

'뻐근한 통증dd, 약함, 등이랑 목으로 뻗침'

In [57]:
#data[data['Character'] == '쥐어짜는 느낌의 통증, NRS 7점, 턱과 목쪽으로 방사되는 통증, 왼팔로도 통증이 뻗치는 듯한 느낌 있음']
#data.iloc[872]['Character']

In [58]:
for m in data.columns[10:21]: #'Experience', 'Character', 'Associated Sx.', 'Factor', 'Event', '약물 투약력', '사회력', '가족력', '외상력', '과거력', '여성력']
  data[m] = data.apply(lambda x : NRS_to_text(x[m]) , axis = 1 )

In [59]:
#data.iloc[872]['Character']

## 9-1) Make a sentence

In [60]:
data['All'] = (data['Chief complaint'].map(str) +'. '+ data['Age'].map(str) + '. '+
                data['Sex'].map(str) + '. '+
                data['Onset'].map(str) + '. '+ data['Location'].map(str) + '. ' +
                data['Duration'].map(str) + '. ' + data['Course'].map(str) + '. ' +
                data['Experience'].map(str) + '. ' + data['Character'].map(str) + '. ' +
                data['Associated Sx.'].map(str)+'. '+ data['Factor'].map(str) + '. ' +
                data['Event'].map(str) + '. ' + data['약물 투약력'].map(str) + '. ' +
                data['사회력'].map(str) + '. ' + data['가족력'].map(str) + '. ' +
                data['외상력'].map(str) + '. ' + data['과거력'].map(str) + '. ' +
                data['여성력'].map(str) + '. ' + data['Obesity']).apply(lambda row: row.strip())

In [61]:
data['All']

0       배가 아파요. 50s. 남자. 1일전. 명치 부위. 지속. 심해짐. 이전에도 3차례...
1       갑자기 배가 너무 아파요. 30s. 남자. 오늘 새벽. 명치에서 시작해 현재 복부 ...
2       속이 쓰려요. 70s. 여자. 1달전. 명치 부위. 지속. 심해짐. . 속이 쓰림,...
3       속이 쓰려요. 10s. 남자. 2달전. 명치부위 . 지속. 심해짐. . 명치부위 압...
4       배가 아파요. 60s. 남자. 1시간전. 명치부위. 지속. 심해짐. . 명치 부위가...
                              ...                        
2210    콜레스테롤이 높대요. 50s. 남자. 1개월 전. . . . 처음. . 피곤, 식욕...
2211    콜레스테롤이 높대요. 30s. 여자. 1주 전. . . . 처음. . 체중변화(/1...
2212    콜레스테롤이 높대요. 60s. 여자. 1개월 전. . . . 처음. . . . . ...
2213    콜레스테롤이 높대요. 60s. 여자. 3개월 전. . . . 처음. . . . . ...
2214    생리를 안해요. 10s. 여자. 처음부터. . . . 처음. 초경 안옴. 턱이 짧음...
Name: All, Length: 2215, dtype: object

## 9) Check spelling

In [62]:
def manual_spelling(x):
  # Need to update more!!
  x = x.replace('Live(', 'Liver(')
  x = x.replace('혈번', '혈변')
  x = x.replace('[회수] 회 / 일', '')
  x = x.replace('ㅊ음', '처음')
  x = x.replace('위엄', '위염')
  x = x.replace('소회기', '소화기')
  x = x.replace('소화쉐양', '소화궤양')
  x = x.replace('무릅', '무릎')
  x = x.replace('성광계', '성관계')
  x = x.replace('게속', '계속')
  x = x.replace('느김', '느낌')
  x = x.replace('불규칙하', '불규칙한')
  x = x.replace('뻗뻗', '뻣뻣')
  x = x.replace('부움', '부음')
  x = x.replace('피료', '피로')
  x = x.replace('들이마쉴', '들이마실')
  x = x.replace('(Olodaterol)', '')
  x = x.replace('체충', '체중')
  x = x.replace('속옥', '속옷')
  x = x.replace('아이업음', '아이없음')
  x = x.replace('재왕절개', '제왕절개')
  x = x.replace('얄', '열')
  x = x.replace('연 (', '열 (')
  x = x.replace('가끔식', '가끔씩')
  x = x.replace('붙성교', '성교')
  x = x.replace('월경량와', '월경량과')
  x = x.replace('호ㅡㅂ음', '호흡음')
  x = x.replace('오료계', '요로계')
  x = x.replace('분철성', '분출성')
  return x

In [63]:
#for m in data.columns[6:21]: #'Experience', 'Character', 'Associated Sx.', 'Factor', 'Event', '약물 투약력', '사회력', '가족력', '외상력', '과거력', '여성력']
#  data[m] = data.apply(lambda x : manual_spelling(x[m]) , axis = 1 )
data['All'] = manual_spelling(data['All'])

In [64]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-r44vzpk1
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-r44vzpk1
  Created wheel for py-hanspell: filename=py_hanspell-1.1-cp37-none-any.whl size=4854 sha256=64292efe3c54c92b9f4fc78a8ab5b006be2b3b6b5906417336886bae744020c8
  Stored in directory: /tmp/pip-ephem-wheel-cache-9vx0z4ac/wheels/0a/25/d1/e5e96476dbb1c318cc26c992dd493394fe42b0c204b3e65588
Successfully built py-hanspell


In [65]:
data['All']

0       배가 아파요. 50s. 남자. 1일전. 명치 부위. 지속. 심해짐. 이전에도 3차례...
1       갑자기 배가 너무 아파요. 30s. 남자. 오늘 새벽. 명치에서 시작해 현재 복부 ...
2       속이 쓰려요. 70s. 여자. 1달전. 명치 부위. 지속. 심해짐. . 속이 쓰림,...
3       속이 쓰려요. 10s. 남자. 2달전. 명치부위 . 지속. 심해짐. . 명치부위 압...
4       배가 아파요. 60s. 남자. 1시간전. 명치부위. 지속. 심해짐. . 명치 부위가...
                              ...                        
2210    콜레스테롤이 높대요. 50s. 남자. 1개월 전. . . . 처음. . 피곤, 식욕...
2211    콜레스테롤이 높대요. 30s. 여자. 1주 전. . . . 처음. . 체중변화(/1...
2212    콜레스테롤이 높대요. 60s. 여자. 1개월 전. . . . 처음. . . . . ...
2213    콜레스테롤이 높대요. 60s. 여자. 3개월 전. . . . 처음. . . . . ...
2214    생리를 안해요. 10s. 여자. 처음부터. . . . 처음. 초경 안옴. 턱이 짧음...
Name: All, Length: 2215, dtype: object

In [66]:
from tqdm.notebook import tqdm
from hanspell import spell_checker

def spelling_check(text):
  #print(text)
  result = spell_checker.check(text) 
  
  return result.as_dict()['checked'] 

In [67]:
for i in tqdm(range(len(data))):
  data['All'][i] = spelling_check(data['All'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [68]:
data['All']

0       배가 아파요. 50s. 남자. 1일 전. 명치 부위. 지속. 심해짐. 이전에도 3차...
1       갑자기 배가 너무 아파요. 30s. 남자. 오늘 새벽. 명치에서 시작해 현재 복부 ...
2       속이 쓰려요. 70s. 여자. 1달 전. 명치 부위. 지속. 심해짐. . 속이 쓰림...
3       속이 쓰려요. 10s. 남자. 2달 전. 명치 부위. 지속. 심해짐. . 명치 부위...
4       배가 아파요. 60s. 남자. 1시간 전. 명치 부위. 지속. 심해짐. . 명치 부...
                              ...                        
2210    콜레스테롤이 높대요. 50s. 남자. 1개월 전. . . . 처음. . 피곤, 식욕...
2211    콜레스테롤이 높대요. 30s. 여자. 1주 전. . . . 처음. . 체중 변화(/...
2212    콜레스테롤이 높대요. 60s. 여자. 1개월 전. . . . 처음. . . . . ...
2213    콜레스테롤이 높대요. 60s. 여자. 3개월 전. . . . 처음. . . . . ...
2214    생리를 안 해요. 10s. 여자. 처음부터. . . . 처음. 초경 안 옴. 턱이 ...
Name: All, Length: 2215, dtype: object

## 10) Change English words to Korean

In [69]:
def replace_eng_to_kor(x):
  # Need to update more!!
  x = x.replace('Liver', '간염')
  x = x.replace('Hepatitis', '간염')
  x = x.replace('DM', '당뇨')
  x = x.replace('TBc', '결핵')
  x = x.replace('TB', '결핵')
  x = x.replace('HTN', '고혈압')
  x = x.replace('Dyslipidemia', '이상지질혈증')
  x = x.replace('dyslipidemia', '이상지질혈증')
  x = x.replace('소염진통제(NSAIDs)', '소염진통제')
  x = x.replace('NSAIDs', '소염진통제')
  x = x.replace('NSAID', '소염진통제')
  x = x.replace('LMP', '마지막 월경')
  x = x.replace("Murphy's sign", '숨을 들이쉬는 동안 오른쪽 갈비뼈 밑 인근 부위를 만지면 통증이 생기는 증상')
  x = x.replace('Valsartan', '고혈압약')
  x = x.replace('고혈압약(amlodipine)', '고혈압약')
  x = x.replace('항고혈압제(Amlodipine)', '고혈압약')
  x = x.replace('Amlodipine', '고혈압약')
  x = x.replace('amlodipine', '고혈압약')
  x = x.replace('Metformin', '비구아니드 계열의 경구용 혈당강하제, 제2형 당뇨병에 사용')
  x = x.replace('BID', '하루에 2번')
  x = x.replace('Bid', '하루에 2번')
  x = x.replace('ciprofloxacin', '항생제')
  x = x.replace('QD', '하루에 한번')
  x = x.replace('우울증약(SSRI)', '우울증약')
  x = x.replace('SSRI', '우울증약')
  x = x.replace('1T', '1알')
  x = x.replace('FAP', '가족성 용종증')
  x = x.replace('COPD', '만성 폐쇄성 폐질환')
  x = x.replace('Roux-Y op', '위우회술')
  x = x.replace('terminal ileum', '직장과 결장 및 소장의 마지막 일부')
  x = x.replace('Hyperlipidemia', '고지혈증')
  x = x.replace('혈압강하제(CCB, ACEi)', '혈압강하제')
  x = x.replace('C-sec', '제왕절개')
  x = x.replace('Non-smoking, Non-Alcohol', '술 담배 안함')
  x = x.replace('F / U / N / D / H / I / S', '빈뇨/절박뇨/야뇨/배뇨통/배뇨지연/요실금/소변줄기 변화')
  
  return x

In [70]:
#for m in data.columns[10:21]: #'Experience', 'Character', 'Associated Sx.', 'Factor', 'Event', '약물 투약력', '사회력', '가족력', '외상력', '과거력', '여성력']
#  data[m] = data.apply(lambda x : replace_eng_to_kor(x[m]) , axis = 1 )

for i in tqdm(range(len(data))):
  data['All'][i] = replace_eng_to_kor(data['All'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


## 11) Change difficuly words to easy words

In [71]:
def replace_diffi_to_easy(x):
  # Need to update more!!
  if x=='지속' or x=='지속 ' or x=='지속됨':
      x = '하루종일'
  x = x.replace('복압 배뇨', '소변을 볼 때 힘을 줘야함')#kidney
  x = x.replace('샅통증', '사타구니 통증')
  x = x.replace('다갈', '몹시 갈증이 남')
  x = x.replace('다음', '몹시 갈증이 나 물을 많이 마심')
  x = x.replace('다뇨', '소변량 증가')
  x = x.replace('뒷무직', '잔변감')
  x = x.replace('요의', '오줌이 마려운 느낌')
  x = x.replace('급성 요폐', '소변이 마려운데 힘줘도 안 나옴')
  x = x.replace('점적뇨', '소변 후 오줌이 몇 방울씩 속옷에 묻어남')
  x = x.replace('두부 외상', '머리를 다침')
  x = x.replace('두부 타격', '머리를 다침')
  x = x.replace('섬망', '환각, 초조함, 떨림')

  x = x.replace('복장뼈아래', '가슴 중앙 아래')#digestive
  x = x.replace('방사통', '퍼지는 통증')
  x = x.replace('반동압통', '눌렀다 떼면 아픔')
  x = x.replace('반발압통', '눌렀다 떼면 아픔')
  x = x.replace('압통점', '누르면 아픔')
  x = x.replace('압통', '누르면 아픔')
  x = x.replace('장음 감소', '장의 운동 소리가 감소')
  x = x.replace('장음감소', '장의 운동 소리가 감소')
  x = x.replace('장음 소실', '장의 운동 소리가 감소')
  x = x.replace('장음 증가', '장의 운동 소리가 증가')
  x = x.replace('장음증가', '장의 운동 소리가 증가')
  x = x.replace('박동성 종괴', '두근거리는 멍울, 혹')
  x = x.replace('종괴', '멍울, 혹')
  x = x.replace('덩이', '멍울, 혹')
  x = x.replace('두덩뼈', '골반뼈 중 일부')
  x = x.replace('공막', '흰자, 눈')
  x = x.replace('혈액응고지연', '피가 안멈춰요')
  x = x.replace('인두통', '목이 아픔')
  x = x.replace('항문 열상', '항문이 찢어져서 생긴 상처')
  x = x.replace('후중감(tenesmus)', '잔변감')
  x = x.replace('경부강직', '목근육이 굳어서 머리를 앞으로 구부릴 수 없음')
  x = x.replace('조음장애', '발음장애')
  x = x.replace('구음장애', '말하기 어려움')
  x = x.replace('추위불내성', '추위를 견디지 못함')
  x = x.replace('추위 불내성', '추위를 견디지 못함')
  x = x.replace('더위불내성', '더위를 견디지 못함')
  x = x.replace('더위 불내성', '더위를 견디지 못함')
  x = x.replace('야간발한', '밤에 땀이 남')
  x = x.replace('야간 발한', '밤에 땀이 남')
  x = x.replace('발한과다', '땀이 많이 남')
  x = x.replace('전신발한', '온몸에 땀이 남')
  x = x.replace('발한', '땀이 남')
  x = x.replace('와파린', '항응고제')
  x = x.replace('경추', '목')
  x = x.replace('요추', '허리')
  x = x.replace('폴립', '종양, 용종')
  x = x.replace('의식 소실', '의식 없어짐')
  x = x.replace('의식소실', '의식 없어짐')
  x = x.replace('황달 소실', '황달 없어짐')
  x = x.replace('유지', '비슷')
  x = x.replace('우상복부', '오른쪽 윗배')
  x = x.replace('우하복부', '오른쪽 아랫배')
  x = x.replace('우측복부', '오른쪽 배')
  x = x.replace('좌측복부', '왼쪽 배')
  x = x.replace('좌하복부', '왼쪽 아랫배')
  x = x.replace('좌상복부', '왼쪽 윗배')
  x = x.replace('하복부통', '아랫배 통증')
  x = x.replace('상복부', '윗배')
  x = x.replace('하복부', '아랫배')
  x = x.replace('심계항진', '심장이 계속 두근거려요')
  x = x.replace('분출성', '뿜어나오는 구토')
  x = x.replace('홍반', '붉은 반점')



  x = x.replace('작열감', '화끈거림')#musculoskeletal
  x = x.replace('광과민성', '빛에 민감')
  x = x.replace('간헐적', '주기적으로')#respiratory
  x = x.replace('간간히', '가끔씩')
  x = x.replace('혈성 콧물', '피가 섞인 콧물')
  x = x.replace('인후통', '목구멍이 아픔')
  x = x.replace('객혈', '피가 섞인 가래')
  x = x.replace('객담', '가래')

  #오목부종 오목 부종 안면부종 다리부종 하지부종 음낭부종 얼굴부종


  x = x.replace('편마비', '한 쪽 마비')#circulator2
  x = x.replace('메티마졸', '갑상선기능항진증 약')
  x = x.replace('희발월경', '월경, 생리 주기가 길다') 
  x = x.replace('측면', '옆')
  x = x.replace('무월경', '월경 안함')
  x = x.replace('지방식', '기름진 음식')
  x = x.replace('위약', '쇠약')
  x = x.replace('쇠약감', '힘이 없음')
  x = x.replace('동공산대', '동공 확대')



  x = x.replace('성교통', '성관계를 할 때 통증')#Para_Female_Child
  x = x.replace('성교', '성관계')
  x = x.replace('회음부', '성기')
  x = x.replace('외음부', '성기')
  x = x.replace('초경(-)', '초경 안함')
  x = x.replace('초경 전', '초경 안함')

  x = x.replace('오심', '구역감')#digestive(jaundice)
  x = x.replace('소양증', '가려움증')
  x = x.replace('쓰리다', '속쓰림')#hematemesis(digestive)
  return x

In [72]:
# for m in data.columns[10:21]: #'Experience', 'Character', 'Associated Sx.', 'Factor', 'Event', '약물 투약력', '사회력', '가족력', '외상력', '과거력', '여성력']
#  data[m] = data.apply(lambda x : replace_diffi_to_easy(x[m]) , axis = 1 )

for i in tqdm(range(len(data))):
  data['All'][i] = replace_diffi_to_easy(data['All'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [73]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2215 entries, 0 to 2214
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   level5/diagnosis  2215 non-null   object 
 1   Chief complaint   2215 non-null   object 
 2   Sex               2215 non-null   object 
 3   Age               2215 non-null   object 
 4   Height            2215 non-null   int64  
 5   Weight            2215 non-null   int64  
 6   Onset             2215 non-null   object 
 7   Location          2215 non-null   object 
 8   Duration          2215 non-null   object 
 9   Course            2215 non-null   object 
 10  Experience        2215 non-null   object 
 11  Character         2215 non-null   object 
 12  Associated Sx.    2215 non-null   object 
 13  Factor            2215 non-null   object 
 14  Event             2215 non-null   object 
 15  약물 투약력            2215 non-null   object 
 16  사회력               2215 non-null   object 


## 12) Erase stopwords using konlpy

In [74]:
!pip install konlpy
import konlpy.tag
okt = konlpy.tag.Okt() # 객체 생성

     |████████████████████████████████| 19.4MB 156kB/s 
     |████████████████████████████████| 92kB 13.2MB/s 
     |████████████████████████████████| 460kB 54.0MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [75]:
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '을'
             '를', '으로', '자', '에', '와', '하다', '요', '다', '.', ',']

In [76]:
def erase_stopwords(text):
  X_train =[]
  temp_x = okt.morphs(text, stem=True)
  #print(temp_x)
  temp_x = [word for word in temp_x if not word in stopwords]
  temp_x = re.findall(r'\w+', str(temp_x))
  X_train.append(temp_x)
  #print(X_train)
  temp_x = ' '.join(map(str, X_train))
  
  return ' '.join(re.findall(r'\w+', str(temp_x)))

In [77]:
#for m in data.columns[10:21]: #'Experience', 'Character', 'Associated Sx.', 'Factor', 'Event', '약물 투약력', '사회력', '가족력', '외상력', '과거력', '여성력']
#  data[m] = data.apply(lambda x : erase_stopwords(x[m]) , axis = 1 )

#data['Chief complaint'] = data.apply(lambda x : erase_stopwords(x['Chief complaint']) , axis = 1 )

for i in tqdm(range(len(data))):
  data['All'][i] = erase_stopwords(data['All'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [80]:
#data.to_csv('processed_DB_5.17.csv', encoding='utf-8-sig')

In [82]:
#import pandas as pd

#data = pd.read_csv('processed_DB_5.17.csv')
#data.drop('Unnamed: 0', axis=1, inplace=True)
#data.fillna('', inplace=True)
#data

# 2. Select features
https://datascience.stackexchange.com/questions/74688/how-to-use-multiple-text-features-for-nlp-classifier

In [83]:
women_data = data.copy()

In [84]:
men_data = data[data['Level1'] != 'para']
men_data = men_data[men_data['level5/diagnosis'] != '폐경증후군']
men_data = men_data[men_data['level5/diagnosis'] != '폐경에 의한 고혈압']

In [85]:
print(len(women_data))
print(len(men_data))

2215
2000


In [86]:
women_train = women_data.copy()
men_train = men_data.copy()

In [87]:
from io import StringIO
col = ['level5/diagnosis', 'All']#, 'Height', 'Weight']
women_df = women_train[col]
men_df = men_train[col]

women_df = women_df[pd.notnull(women_df['All'])]
men_df = men_df[pd.notnull(men_df['All'])]

#df['category_id'] = df['level5/diagnosis'].factorize()[0]
#category_id_df = df[['level5/diagnosis', 'category_id']].drop_duplicates().sort_values('category_id')
#category_to_id = dict(category_id_df.values)
#id_to_category = dict(category_id_df[['category_id', 'level5/diagnosis']].values)

men_df.head()

,level5/diagnosis,All
0,소화성 궤양,배 아프다 50 s 남자 1일 전 명치 부위 지속 심해지다 이전 에도 3 차례 통증...
1,소화성 궤양,갑자기 배 너무 아프다 30 s 남자 오늘 새벽 명치 에서 시작 현재 복부 전체 통...
2,소화성 궤양,속이다 쓰리다 70 s 여자 1 달 전 명치 부위 지속 심해지다 속이다 쓰리다 약하...
3,소화성 궤양,속이다 쓰리다 10 s 남자 2 달 전 명치 부위 지속 심해지다 명치 부위 누르다 ...
4,소화성 궤양,배 아프다 60 s 남자 1시간 전 명치 부위 지속 심해지다 명치 부위 아프다 쓰리...


In [88]:
#df.to_csv('digestive.csv', index=False)

In [89]:
from sklearn.model_selection import train_test_split

w_ori_X_train, w_ori_X_test, w_ori_y_train, w_ori_y_test = train_test_split(women_df, women_df['level5/diagnosis'], random_state = 0)
m_ori_X_train, m_ori_X_test, m_ori_y_train, m_ori_y_test = train_test_split(men_df, men_df['level5/diagnosis'], random_state = 0)

In [90]:
#!pip3 install tensorflow_addons

# Tokenizer: LMKor

In [91]:
#!pip install kobert_transformers
#!pip install sentencepiece
!pip install transformers

     |████████████████████████████████| 2.3MB 7.2MB/s 
     |████████████████████████████████| 3.3MB 36.3MB/s 
     |████████████████████████████████| 901kB 44.4MB/s 


In [92]:
# funnel: https://github.com/kiyoungkim1/LMkor
from transformers import FunnelTokenizerFast, FunnelModel
tokenizer_funnel = FunnelTokenizerFast.from_pretrained("kykim/funnel-kor-base")
tokenizer_funnel

PreTrainedTokenizerFast(name_or_path='kykim/funnel-kor-base', vocab_size=42000, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '<sep>', 'pad_token': '<pad>', 'cls_token': '<cls>', 'mask_token': '<mask>'})

In [93]:
#!pip install konlpy
#import konlpy.tag
#okt = konlpy.tag.Okt() # 객체 생성

#stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '을'
#             '를', '으로', '자', '에', '와', '한', '하다', '요']

In [94]:
def only_letters_num(x):
  #x = okt.morphs(x, stem=True)
  #x = [word for word in x if not word in stopwords]
  return ' '.join(re.findall(r'\w+', str(x)))

In [95]:
import re

w_ori_X_train2 = w_ori_X_train.copy()
w_ori_X_test2 = w_ori_X_test.copy()

w_ori_X_train2['All'] = w_ori_X_train.apply(lambda x : only_letters_num(x['All']) , axis = 1 )
w_ori_X_test2['All'] = w_ori_X_test.apply(lambda x : only_letters_num(x['All']) , axis = 1 )

w_document_bert_train = ["[CLS] " + str(s) + " [SEP]" for s in w_ori_X_train2['All']]
w_document_bert_test = ["[CLS] " + str(s) + " [SEP]" for s in w_ori_X_test2['All']]

w_ko_tokenized_texts_train = [tokenizer_funnel.tokenize(s) for s in w_document_bert_train]
w_ko_tokenized_texts_test = [tokenizer_funnel.tokenize(s) for s in w_document_bert_test]

##############
m_ori_X_train2 = m_ori_X_train.copy()
m_ori_X_test2 = m_ori_X_test.copy()

m_ori_X_train2['All'] = m_ori_X_train.apply(lambda x : only_letters_num(x['All']) , axis = 1 )
m_ori_X_test2['All'] = m_ori_X_test.apply(lambda x : only_letters_num(x['All']) , axis = 1 )

m_document_bert_train = ["[CLS] " + str(s) + " [SEP]" for s in m_ori_X_train2['All']]
m_document_bert_test = ["[CLS] " + str(s) + " [SEP]" for s in m_ori_X_test2['All']]

m_ko_tokenized_texts_train = [tokenizer_funnel.tokenize(s) for s in m_document_bert_train]
m_ko_tokenized_texts_test = [tokenizer_funnel.tokenize(s) for s in m_document_bert_test]

In [96]:
print(w_ori_X_train['All'][0])
print(w_ori_X_train2['All'][0])

배 아프다 50 s 남자 1일 전 명치 부위 지속 심해지다 이전 에도 3 차례 통증 이번 보다 약하다 칼 로 찢기다 듯 한 통증 심하다 퍼지다 통증 등 퍼지다 구토 속 쓰리다 응급 어지러움 갈증 소 변량 감소 자세 따르다 변화 오른쪽 돌 아누 우 면 완화 반듯이 누우 면 악화 술 1 주일 6 7 번 하루 소주 2 3 병 담배 30 갑년 식사 매우 불규칙 직업 포클레인 조종수 과체중
배 아프다 50 s 남자 1일 전 명치 부위 지속 심해지다 이전 에도 3 차례 통증 이번 보다 약하다 칼 로 찢기다 듯 한 통증 심하다 퍼지다 통증 등 퍼지다 구토 속 쓰리다 응급 어지러움 갈증 소 변량 감소 자세 따르다 변화 오른쪽 돌 아누 우 면 완화 반듯이 누우 면 악화 술 1 주일 6 7 번 하루 소주 2 3 병 담배 30 갑년 식사 매우 불규칙 직업 포클레인 조종수 과체중


In [97]:
print(max(len(l) for l in w_ko_tokenized_texts_train))
print(max(len(l) for l in m_ko_tokenized_texts_train))

162
162


In [128]:
from keras.preprocessing.text import Tokenizer

w_max_words = 1800 #Choose only 1800 words which has high frequency
w_tokenizer = Tokenizer(num_words = w_max_words)
w_tokenizer.fit_on_texts(w_ko_tokenized_texts_train)

w_X_train_sequence = w_tokenizer.texts_to_sequences(w_ko_tokenized_texts_train)
w_X_test_sequence = w_tokenizer.texts_to_sequences(w_ko_tokenized_texts_test)

w_y_train_dummies = pd.get_dummies(women_df['level5/diagnosis']).loc[w_ori_X_train.index]
w_y_test_dummies = pd.get_dummies(women_df['level5/diagnosis']).loc[w_ori_X_test.index]

from keras.preprocessing.sequence import pad_sequences
w_max_len = max(len(l) for l in w_ko_tokenized_texts_train)
w_X_train_sequence = pad_sequences(w_X_train_sequence, maxlen=w_max_len)
w_X_test_sequence = pad_sequences(w_X_test_sequence, maxlen=w_max_len)

###############
m_max_words = 1600
m_tokenizer = Tokenizer(num_words = m_max_words)
m_tokenizer.fit_on_texts(m_ko_tokenized_texts_train)

m_X_train_sequence = m_tokenizer.texts_to_sequences(m_ko_tokenized_texts_train)
m_X_test_sequence = m_tokenizer.texts_to_sequences(m_ko_tokenized_texts_test)

m_y_train_dummies = pd.get_dummies(men_df['level5/diagnosis']).loc[m_ori_X_train.index]
m_y_test_dummies = pd.get_dummies(men_df['level5/diagnosis']).loc[m_ori_X_test.index]

m_max_len = max(len(l) for l in m_ko_tokenized_texts_train)
m_X_train_sequence = pad_sequences(m_X_train_sequence, maxlen=m_max_len)
m_X_test_sequence = pad_sequences(m_X_test_sequence, maxlen=m_max_len)

In [129]:
print(len(w_y_train_dummies.columns))
print(len(m_y_test_dummies.columns))

225
193


In [130]:
print(len(w_tokenizer.index_word) + 1)
print(len(w_tokenizer.word_counts))

print(len(m_tokenizer.index_word) + 1)
print(len(m_tokenizer.word_counts))

1829
1828
1670
1669


In [131]:
print(w_tokenizer.num_words)
print(m_tokenizer.num_words)

1800
1600


In [132]:
print(w_max_len)
print(m_max_len)

162
162


In [133]:
print(len(women_df['level5/diagnosis'].unique()))
print(len(men_df['level5/diagnosis'].unique()))

225
193


# class weight

In [134]:
from sklearn.utils import class_weight
import numpy as np

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(women_df['level5/diagnosis']),
                                                 women_df['level5/diagnosis'])

w_class_weights = dict(enumerate(class_weights))

In [135]:
from sklearn.utils import class_weight
import numpy as np

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(men_df['level5/diagnosis']),
                                                 men_df['level5/diagnosis'])

m_class_weights = dict(enumerate(class_weights))

# Model

## Multi (LSTM + Conv) _W

In [136]:
def w_top_k_hitrate(predicted_proba, n=2):

  eval_metric = []
  top_k_result = tf.math.top_k(predicted_proba, k=n, sorted=True)
  for i in range(len(top_k_result[1])):
    first = top_k_result[1][i][0]
    second = top_k_result[1][i][1]
    third = top_k_result[1][i][2]

    if w_ori_X_test['level5/diagnosis'].values[i] in (w_y_test_dummies.columns[first],
                                                      w_y_test_dummies.columns[second],
                                                      w_y_test_dummies.columns[third]):
      eval_metric.append(1)
    else:
      eval_metric.append(0)

  return np.mean(eval_metric)

In [146]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPool1D, Concatenate, Flatten, Dropout, Dense, Conv1D
from keras.layers import MaxPool1D, BatchNormalization, SimpleRNN, MaxPooling1D
from keras.models import Model
from keras.layers import Embedding, Dense, LSTM, GlobalMaxPooling1D, Dropout, Conv1D, Bidirectional
from keras.models import Sequential, load_model, save_model
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint

w_VOCAB_SIZE = len(w_tokenizer.index_word) + 1
num_filters = 512
filter_sizes = [5, 7, 9]

inputs = Input(shape=(w_max_len,), dtype='int32')
embedding = Embedding(w_VOCAB_SIZE, 1000)(inputs) #800하면 top_k가 0.01높지만 나머지는 0.01낮다.
print(embedding.shape)

lstm_1 = LSTM(512, dropout=0.3, return_sequences=True)(embedding)
lstm_2 = LSTM(512, dropout=0.3, return_sequences=True)(embedding)
lstm_3 = LSTM(512, dropout=0.3, return_sequences=True)(embedding)

lstm_maxpool_0 = MaxPooling1D(pool_size=4)(lstm_1)
lstm_maxpool_1 = MaxPooling1D(pool_size=4)(lstm_2)
lstm_maxpool_2 = MaxPooling1D(pool_size=4)(lstm_3)

n_conv_0 = Conv1D(num_filters, kernel_size=(filter_sizes[0]),
                padding='valid', activation='relu')(lstm_maxpool_0)
n_conv_1 = Conv1D(num_filters, kernel_size=(filter_sizes[1]),
                padding='valid', activation='relu')(lstm_maxpool_1)
n_conv_2 = Conv1D(num_filters, kernel_size=(filter_sizes[2]),
                padding='valid', activation='relu')(lstm_maxpool_2)

#MaxPool1D
n_maxpool_0 = GlobalMaxPooling1D()(n_conv_0)
n_maxpool_1 = GlobalMaxPooling1D()(n_conv_1)
n_maxpool_2 = GlobalMaxPooling1D()(n_conv_2)

##############
conv_0 = Conv1D(num_filters, kernel_size=(filter_sizes[0]),
                padding='valid', activation='relu')(embedding)
conv_1 = Conv1D(num_filters, kernel_size=(filter_sizes[1]),
                padding='valid', activation='relu')(embedding)
conv_2 = Conv1D(num_filters, kernel_size=(filter_sizes[2]),
                padding='valid', activation='relu')(embedding)

#MaxPool1D
maxpool_0 = GlobalMaxPooling1D()(conv_0)
maxpool_1 = GlobalMaxPooling1D()(conv_1)
maxpool_2 = GlobalMaxPooling1D()(conv_2)
################

concatenated = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2,
                                    n_maxpool_0, n_maxpool_1, n_maxpool_2])
flatten_layer = Flatten()(concatenated)
dropout_layer1 = Dropout(0.5)(flatten_layer)

dense_layer3 = Dense(units=2400, activation='relu')(dropout_layer1)
dropout_layer4 = Dropout(0.5)(dense_layer3)

dense_layer4 = Dense(units=800, activation='relu')(dropout_layer4)
dropout_layer5 = Dropout(0.5)(dense_layer4)

output = Dense(units=len(women_df['level5/diagnosis'].unique()),
               activation='softmax')(dropout_layer5)

w_model = Model(inputs=inputs, outputs=output)

w_model.compile(optimizer='adam', loss='categorical_crossentropy',
                metrics=['accuracy',
                         tf.keras.metrics.Precision(name='precision')])

history = w_model.fit(w_X_train_sequence, w_y_train_dummies, epochs=100,
                      batch_size = 512, class_weight = w_class_weights,
                      validation_split=0.2)

(None, 162, 1000)
Epoch 1/100
3/3 [==============================] - 15s 3s/step - loss: 5.4358 - accuracy: 0.0042 - precision: 0.0000e+00 - val_loss: 5.4625 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 7s 2s/step - loss: 5.5644 - accuracy: 0.0101 - precision: 0.0000e+00 - val_loss: 5.4863 - val_accuracy: 0.0120 - val_precision: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 7s 2s/step - loss: 5.4341 - accuracy: 0.0057 - precision: 0.0000e+00 - val_loss: 5.4589 - val_accuracy: 0.0030 - val_precision: 0.0000e+00
Epoch 4/100
3/3 [==============================] - 8s 2s/step - loss: 5.4234 - accuracy: 0.0091 - precision: 0.0000e+00 - val_loss: 5.4530 - val_accuracy: 0.0030 - val_precision: 0.0000e+00
Epoch 5/100
3/3 [==============================] - 8s 2s/step - loss: 5.4396 - accuracy: 0.0061 - precision: 0.0000e+00 - val_loss: 5.4667 - val_accuracy: 0.0120 - val_precision: 0.0000e+00
Epoch 6/100
3/3 [==========

In [147]:
test_result = w_model.evaluate(w_X_test_sequence, w_y_test_dummies)

print("Accuracy of Test: {:.4f}".format(test_result[1])) #(TP+TN)/(TP+FN+FP+TN)
print("Precision of Test: {:.4f}".format(test_result[2])) #(TP/(TP+FP))
#print("F1-score of Test: {:.4f}".format(np.mean(test_result[3])))
print("top_k_result: {:.4f}".format(w_top_k_hitrate(w_model.predict(w_X_test_sequence), n=3)))

18/18 [==============================] - 1s 68ms/step - loss: 1.0912 - accuracy: 0.7978 - precision: 0.9516
Accuracy of Test: 0.7978
Precision of Test: 0.9516
top_k_result: 0.9097


In [ ]:
#dropout 0.3
Accuracy of Test: 0.8195
Precision of Test: 0.9523
top_k_result: 0.9152

#Accuracy of Test: 0.8649
#Precision of Test: 0.8894
#top_k_result: 0.9272

In [148]:
top_k_result = tf.math.top_k(w_model.predict(w_X_test_sequence), k=1, sorted=True)
first = top_k_result[1]
b = w_y_test_dummies.columns[first]

w_ori_y_test = np.asarray(w_ori_y_test)

compare_result = pd.concat([pd.DataFrame(w_ori_y_test, columns=['True']).reset_index(drop=True),
                            pd.DataFrame(b, columns=['predict']).reset_index(drop=True)],
                           axis=1)

misclassified = np.where(compare_result['True'] != compare_result['predict'])
compare_result.iloc[misclassified]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,True,predict
26,커피 / 약물에 의한 요붕증,요폐색 해소 후 요붕증
27,폐경증후군,발작심실상성 빈맥
28,척추관 협착증,연골연화증
30,폐색전증,대동맥 박리
35,심부전,악성 고혈암
...,...,...
538,담도암 / 췌장 두부암,담관염
543,폐렴,상기도출혈
546,약물 / 알콜에 의한 증상,신경인성 방광
547,유방낭종,섬유 낭종성 질환


## Multi (LSTM -> Conv) _M

In [149]:
def m_top_k_hitrate(predicted_proba, n=2):

  eval_metric = []
  top_k_result = tf.math.top_k(predicted_proba, k=n, sorted=True)
  for i in range(len(top_k_result[1])):
    first = top_k_result[1][i][0]
    second = top_k_result[1][i][1]
    third = top_k_result[1][i][2]

    if m_ori_X_test['level5/diagnosis'].values[i] in (m_y_test_dummies.columns[first],
                                                      m_y_test_dummies.columns[second],
                                                      m_y_test_dummies.columns[third]):
      eval_metric.append(1)
    else:
      eval_metric.append(0)

  return np.mean(eval_metric)

In [ ]:
from keras.layers import MaxPooling1D
m_VOCAB_SIZE = len(m_tokenizer.index_word) + 1
num_filters = 512
filter_sizes = [5, 7, 9]

inputs = Input(shape=(m_max_len,), dtype='int32')
embedding = Embedding(m_VOCAB_SIZE, 1000)(inputs) #800하면 top_k가 0.01높지만 나머지는 0.01낮다.
print(embedding.shape)

#lstm_1 = LSTM(512, dropout=0.2, return_sequences=True)(embedding)
#lstm_2 = LSTM(512, dropout=0.2, return_sequences=True)(embedding)
#lstm_3 = LSTM(512, dropout=0.2, return_sequences=True)(embedding)

lstm_1 = LSTM(512, dropout=0.3, return_sequences=True)(embedding)
lstm_2 = LSTM(512, dropout=0.3, return_sequences=True)(embedding)
lstm_3 = LSTM(512, dropout=0.3, return_sequences=True)(embedding)

lstm_maxpool_0 = MaxPooling1D(pool_size=4)(lstm_1)
lstm_maxpool_1 = MaxPooling1D(pool_size=4)(lstm_2)
lstm_maxpool_2 = MaxPooling1D(pool_size=4)(lstm_3)

n_conv_0 = Conv1D(num_filters, kernel_size=(filter_sizes[0]),
                padding='valid', activation='relu')(lstm_maxpool_0)
n_conv_1 = Conv1D(num_filters, kernel_size=(filter_sizes[1]),
                padding='valid', activation='relu')(lstm_maxpool_1)
n_conv_2 = Conv1D(num_filters, kernel_size=(filter_sizes[2]),
                padding='valid', activation='relu')(lstm_maxpool_2)

#MaxPool1D
n_maxpool_0 = GlobalMaxPooling1D()(n_conv_0)
n_maxpool_1 = GlobalMaxPooling1D()(n_conv_1)
n_maxpool_2 = GlobalMaxPooling1D()(n_conv_2)

##############
conv_0 = Conv1D(num_filters, kernel_size=(filter_sizes[0]),
                padding='valid', activation='relu')(embedding)
conv_1 = Conv1D(num_filters, kernel_size=(filter_sizes[1]),
                padding='valid', activation='relu')(embedding)
conv_2 = Conv1D(num_filters, kernel_size=(filter_sizes[2]),
                padding='valid', activation='relu')(embedding)

#MaxPool1D
maxpool_0 = GlobalMaxPooling1D()(conv_0)
maxpool_1 = GlobalMaxPooling1D()(conv_1)
maxpool_2 = GlobalMaxPooling1D()(conv_2)
################

concatenated = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2,
                                    n_maxpool_0, n_maxpool_1, n_maxpool_2])
flatten_layer = Flatten()(concatenated)
dropout_layer1 = Dropout(0.5)(flatten_layer)

dense_layer3 = Dense(units=2400, activation='relu')(dropout_layer1)
dropout_layer4 = Dropout(0.5)(dense_layer3)

dense_layer4 = Dense(units=800, activation='relu')(dropout_layer4)
dropout_layer5 = Dropout(0.5)(dense_layer4)

output = Dense(units=len(men_df['level5/diagnosis'].unique()),
               activation='softmax')(dropout_layer5)

m_model = Model(inputs=inputs, outputs=output)

m_model.compile(optimizer='adam', loss='categorical_crossentropy',
                metrics=['accuracy',
                         tf.keras.metrics.Precision(name='precision')])

history = m_model.fit(m_X_train_sequence, m_y_train_dummies, epochs=100,
                      batch_size = 216, class_weight = m_class_weights,
                      validation_split=0.2)

(None, 162, 1000)
Epoch 1/100
6/6 [==============================] - 17s 2s/step - loss: 5.2993 - accuracy: 0.0024 - precision: 0.0000e+00 - val_loss: 5.2782 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 2/100
6/6 [==============================] - 7s 1s/step - loss: 5.2997 - accuracy: 0.0134 - precision: 0.0000e+00 - val_loss: 5.2946 - val_accuracy: 0.0033 - val_precision: 0.0000e+00
Epoch 3/100
6/6 [==============================] - 7s 1s/step - loss: 5.1048 - accuracy: 0.0066 - precision: 0.0000e+00 - val_loss: 5.2573 - val_accuracy: 0.0067 - val_precision: 0.0000e+00
Epoch 4/100
6/6 [==============================] - 7s 1s/step - loss: 5.1912 - accuracy: 0.0053 - precision: 0.0000e+00 - val_loss: 5.2626 - val_accuracy: 0.0033 - val_precision: 0.0000e+00
Epoch 5/100
6/6 [==============================] - 7s 1s/step - loss: 5.0714 - accuracy: 0.0097 - precision: 0.0000e+00 - val_loss: 5.2434 - val_accuracy: 0.0033 - val_precision: 0.0000e+00
Epoch 6/100
6/6 [==========

In [153]:
test_result = m_model.evaluate(m_X_test_sequence, m_y_test_dummies)

print("Accuracy of Test: {:.4f}".format(test_result[1])) #(TP+TN)/(TP+FN+FP+TN)
print("Precision of Test: {:.4f}".format(test_result[2])) #(TP/(TP+FP))
#print("F1-score of Test: {:.4f}".format(np.mean(test_result[3])))
print("top_k_result: {:.4f}".format(m_top_k_hitrate(m_model.predict(m_X_test_sequence), n=3)))

16/16 [==============================] - 1s 71ms/step - loss: 1.0111 - accuracy: 0.8240 - precision: 0.9553
Accuracy of Test: 0.8240
Precision of Test: 0.9553
top_k_result: 0.9060


In [154]:
top_k_result = tf.math.top_k(m_model.predict(m_X_test_sequence), k=1, sorted=True)
first = top_k_result[1]
b = m_y_test_dummies.columns[first]

m_ori_y_test = np.asarray(m_ori_y_test)

compare_result = pd.concat([pd.DataFrame(m_ori_y_test, columns=['True']).reset_index(drop=True),
                            pd.DataFrame(b, columns=['predict']).reset_index(drop=True)],
                           axis=1)

misclassified = np.where(compare_result['True'] != compare_result['predict'])
compare_result.iloc[misclassified]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,True,predict
8,용혈성 요독증후군,연쇄상구균 감염 후 사구체신염
9,급성 췌장염,허혈성 대장염
15,급성 신우신염,연쇄상구균 감염 후 사구체신염
20,횡문근융해증,요로 감염
28,목 삠,요추염좌
...,...,...
471,소화성 궤양,저혈당
476,당뇨병,절박성 요실금
485,난소 염전,자궁외임신
491,신경 이상에 의한 변비,약물에 의한 변비


# Save Models

In [155]:
import pickle

# Save processed disease names
with open("ver6_men_diseases.txt", "wb") as fp:
  pickle.dump(m_y_test_dummies.columns, fp)

with open("ver6_women_diseases.txt", "wb") as fp:
  pickle.dump(w_y_test_dummies.columns, fp)

# Save tokenizer
with open('ver6_men_tokenizer.pickle', 'wb') as handle:
    pickle.dump(m_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('ver6_women_tokenizer.pickle', 'wb') as handle:
    pickle.dump(w_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save ML model
#with open('ml_random_forest.pkl', 'wb') as file:
#    pickle.dump(ml_model, file)

# Save DL model
## 1
m_model.save('ver6_men_model.h5')
w_model.save('ver6_women_model.h5')

## 2
#model_json = model.to_json()
#with open("baseline_model.json", "w") as json_file : 
#    json_file.write(model_json)

#model.save_weights("baseline_model_weights.h5")

# For test

In [ ]:
!pip install transformers
!pip install konlpy
from transformers import FunnelTokenizerFast
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
import pickle
import konlpy.tag
import re

In [156]:
def to_nan(x):
  if(x == '-'):
    x = ''
  elif(x == '아니오'):
    x = ''
  elif(x == '아뇨'):
    x = ''
  elif(x == '몰라요'):
    x = ''
  elif(x == '모릅니다'):
    x = ''
  elif(x == '모름'):
    x = ''
  elif(x == '아뇨'):
    x = ''
  elif(x == '없습니다'):
    x = ''
  elif(x == '없어요'):
    x = ''
  elif(x == '없음'):
    x = ''
  elif(x == '.'):
    x = ''    
  return x

def NRS_to_text(text):

  a = re.findall('NRS.*?점', text) #NRS 5점
  if len(a) == 0:
    a = re.findall('NRS.*?\d~\d', text) #NRS 4~6 / NRS : 4~6
  if len(a) == 0:
    a = re.findall('NRS.*?\d', text) #NRS: 5
  
  try:
    t = re.findall('[0-9]+', a[0])

    if (len(t) == 2):
      score = (int(t[0])+int(t[1]))/ 2
    else:
      score = int(t[0])

    if score >= 7:
      to_text = '심함'
    elif score >= 4:
      to_text = '중간'
    else:
      to_text = '약함'

    text = text.replace(a[0], to_text)
  
  except:
    pass

  return text

from hanspell import spell_checker

def spelling_check(text):
  #print(text)
  result = spell_checker.check(text) 
  
  return result.as_dict()['checked'] 

def define_obesity(x):
  if x < 0:
    return '알 수 없음'
  elif x < 20.0:
    return '저체중'
  elif x <= 24.0:
    return '정상'
  elif x <= 29.0:
    return '과체중'
  else:
    return '비만'


def only_letters_num(x):
  return ' '.join(re.findall(r'\w+', str(x)))


okt = konlpy.tag.Okt() # 객체 생성
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '을'
             '를', '으로', '자', '에', '와', '하다', '요', '다', '.', ',']
             
def erase_stopwords(text):
  temp_x = okt.morphs(text, stem=True)
  temp_x = [word for word in temp_x if not word in stopwords]
  temp_x = re.findall(r'\w+', str(temp_x))
  temp_x = ' '.join(map(str, temp_x))
  
  return ' '.join(re.findall(r'\w+', str(temp_x)))

In [157]:
with open('ver6_men_tokenizer.pickle', 'rb') as handle:
    m_tokenizer = pickle.load(handle)

m_loaded_model = load_model('ver6_men_model.h5')

with open("ver6_men_diseases.txt", "rb") as fp:
    m_disease_codes = pickle.load(fp)

In [158]:
test_df = {
  'Height': '180',
  'Weight': '75',
  'Age': '30s',
  'Chief complaint': '배가 아파요',
  'Sex': '남자',
  'Onset': '1일 전',
  'Location': '명치 부위',
  'Duration' : '지속',
  'Course': '심해짐',
  'Experience' : '이전에도 3차례, 통증은 이번보다 약했음',
  'Character': '칼로 찢기는 듯한 통증, NRS 8점, 방사통: 등으로 퍼짐',
  'Associated Sx.': '구토, 속쓰림, 어지러움, 갈증, 소변량 감소',
  'Factor': '오른쪽으로 돌아 누우면 완화',
  'Event': '',
  '약물 투약력': '',
  '사회력': '술: 1주일 6~7번, 식사는 매우 불규칙',
  '가족력': '',
  '외상력': '',
  '과거력': '',
  '여성력': ''
}

data = pd.DataFrame([test_df])

In [159]:
data.values

array([['180', '75', '30s', '배가 아파요', '남자', '1일 전', '명치 부위', '지속', '심해짐',
        '이전에도 3차례, 통증은 이번보다 약했음', '칼로 찢기는 듯한 통증, NRS 8점, 방사통: 등으로 퍼짐',
        '구토, 속쓰림, 어지러움, 갈증, 소변량 감소', '오른쪽으로 돌아 누우면 완화', '', '',
        '술: 1주일 6~7번, 식사는 매우 불규칙', '', '', '', '']], dtype=object)

In [160]:
data

,Height,Weight,Age,Chief complaint,Sex,Onset,Location,Duration,Course,Experience,Character,Associated Sx.,Factor,Event,약물 투약력,사회력,가족력,외상력,과거력,여성력
0,180,75,30s,배가 아파요,남자,1일 전,명치 부위,지속,심해짐,"이전에도 3차례, 통증은 이번보다 약했음","칼로 찢기는 듯한 통증, NRS 8점, 방사통: 등으로 퍼짐","구토, 속쓰림, 어지러움, 갈증, 소변량 감소",오른쪽으로 돌아 누우면 완화,,,"술: 1주일 6~7번, 식사는 매우 불규칙",,,,


In [161]:
# Handle Nan values 
data = data.fillna('-')

for i in range(len(data.columns)):
  data[data.columns[i]] = data.apply(lambda x : to_nan(x[data.columns[i]]) , axis = 1 )

# Define obesity by calculating BMI with Height and Weight
data['Height'] = data['Height'].replace('', '0')
data['Weight'] = data['Weight'].replace('', '0')

data['Height'] = data['Height'].astype('int')
data['Weight'] = data['Weight'].astype('int')

data['BMI'] = data['Weight'] / (data['Height']/100)**2
data['BMI'].fillna(-1, inplace=True)
data['Obesity'] = define_obesity(data['BMI'].values)
  
# Change Age to Groups of AgeX(-> Since we are going to receive their age as a group_format)
data['Age'] = data['Age'].astype(str)

In [162]:
# Make a sentence
data['All'] = (data['Chief complaint'].values +'. '+ data['Age'].values + '. '+
                  data['Onset'].values + '. '+ data['Location'].values + '. ' +
                  data['Sex'].values + '. '+
                  data['Duration'].values + '. ' + data['Course'].values + '. ' +
                  data['Experience'].values + '. ' + data['Character'].values + '. ' +
                  data['Associated Sx.'].values+'. '+ data['Factor'].values + '. ' +
                  data['Event'].values + '. ' + data['약물 투약력'].values + '. ' +
                  data['사회력'].values + '. ' + data['가족력'].values + '. ' +
                  data['외상력'].values + '. ' + data['과거력'].values + '. ' +
                  data['여성력'].values + '. ' + data['Obesity'].values)

In [163]:
data

,Height,Weight,Age,Chief complaint,Sex,Onset,Location,Duration,Course,Experience,Character,Associated Sx.,Factor,Event,약물 투약력,사회력,가족력,외상력,과거력,여성력,BMI,Obesity,All
0,180,75,30s,배가 아파요,남자,1일 전,명치 부위,지속,심해짐,"이전에도 3차례, 통증은 이번보다 약했음","칼로 찢기는 듯한 통증, NRS 8점, 방사통: 등으로 퍼짐","구토, 속쓰림, 어지러움, 갈증, 소변량 감소",오른쪽으로 돌아 누우면 완화,,,"술: 1주일 6~7번, 식사는 매우 불규칙",,,,,23.148148,정상,배가 아파요. 30s. 1일 전. 명치 부위. 남자. 지속. 심해짐. 이전에도 3차...


In [164]:
# Change NRS to text
data['All'] = NRS_to_text(data['All'].values[0])

# Spelling Check
data['All'] = spelling_check(data['All'].values[0])

# Erase stopwords using konlpy
data['All'] = erase_stopwords(data['All'].values[0])

In [167]:
data['All'] = only_letters_num(data['All'].values)
document_bert_data = ["[CLS] " + str(s) + " [SEP]" for s in data['All'].values]

In [168]:
document_bert_data

['[CLS] 배 아프다 30 s 1일 전 명치 부위 남자 지속 심해지다 이전 에도 3 차례 통증 이번 보다 약하다 칼 로 찢기다 듯 한 통증 심하다 방사 통 등 퍼지다 구토 속 쓰리다 어지러움 갈증 소 변량 감소 오른쪽 돌 아누 우 면 완화 술 1 주일 6 7 번 식사 매우 불규칙 정상 [SEP]']

In [169]:
m_max_len = 162
tokenizer_funnel = FunnelTokenizerFast.from_pretrained("kykim/funnel-kor-base")

if data['Sex'].values =='남자':
  ko_tokenized_texts_data = [tokenizer_funnel.tokenize(s) for s in document_bert_data]
  data_sequence = m_tokenizer.texts_to_sequences(ko_tokenized_texts_data)
  data_sequence = pad_sequences(data_sequence, maxlen = m_max_len).reshape(1,-1)

In [170]:
y_prob = m_loaded_model.predict(data_sequence)

In [171]:
top_k_result = tf.math.top_k(y_prob, k=3, sorted=True)

first = top_k_result[1][0][0], top_k_result.values.numpy()[0][0]
second = top_k_result[1][0][1], top_k_result.values.numpy()[0][1]
third = top_k_result[1][0][2], top_k_result.values.numpy()[0][2]

first_pred_disease_name = m_disease_codes[first[0]]
second_pred_disease_name = m_disease_codes[second[0]]
third_pred_disease_name = m_disease_codes[third[0]]

In [173]:
aa = pd.read_csv('Disease_info.csv')
#aa.drop('Unnamed: 0', axis=1, inplace=True)
aa

,원 질병이름,사이트 질병 이름,진료과,동의어,정의,*비고,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,ACEI(혈압강하제) 복용으로 인한 증상,(ACEI side effect),순환기내과,ACEI 부작용,혈압 강하제 중 ACEI 복용 후 나타나는 부작용을 의미합니다.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,간염,간염(Hepatitis),소화기내과,"간세포 염증,간염증",간염은 간세포 조직에 염증이 생긴 것을 의미합니다.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,간질성 폐질환,미만성 간질성 폐질환(Diffuse Interstitial lung disease),호흡기내과,"간질성 폐질환,미만성 침윤성 폐질환,특발성 간질성 폐렴,폐섬유증,폐섬유화증","미만성 간질성 폐질환(DILD)은 폐간질에 침범되는 비종양성, 비감염성 질환을 총칭...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,갈비연골염,갈비연골염,흉부외과,늑연골염,갈비연골염은 늑골과 흉골을 연결하는 갈비 연골에 염증이 생긴 것입니다.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,감염성 관절염,감염성 관절염,류미티스과,관절염,감염성 관절염은 보통 박테리아나 경우에 따라 바이러스 또는 진균으로 인해 관절액과 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,지주막하 출혈,업데이트 예정 (전처리 중 drop 질병),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,커피 / 약물에 의한 요붕증,업데이트 예정 (전처리 중 drop 질병),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225,터너증후군,업데이트 예정 (전처리 중 drop 질병),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
226,하지신경 손상,업데이트 예정 (전처리 중 drop 질병),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [174]:
first_info = aa[aa['원 질병이름'] == first_pred_disease_name]
print(first_pred_disease_name, round(first[1]*100, 2),'%','\n',
      '동의어:', first_info['동의어'].values[0]+'\n',
      '진료과: ', first_info['진료과'].values[0]+'\n',
      '질병 설명:', first_info['정의'].values[0]
      )

소화성 궤양 99.74 % 
 동의어: 소화성 궤양,십이지장 궤양
 진료과:  소화기내과, 노년내과
 질병 설명: 위궤양은 위점막이 헐어서 궤양이 점막뿐만 아니라 근육층까지 침범한 것을 말합니다.


In [175]:
first_info = aa[aa['원 질병이름'] == first_pred_disease_name]
first_info['진료과'].values[0]

'소화기내과, 노년내과'